In [ ]:
import pandas as pd
from sklearn.metrics import mutual_info_score
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
link = 'https://raw.githubusercontent.com/61050960/Dataset/main/pima-indians-diabetes1.csv?raw=true'
df = pd.read_csv(link)
df

,Number of times pregnant,Plasma glucose concentration a 2 hours in an oral glucose tolerance test,Diastolic blood pressure (mm Hg),Triceps skin fold thickness (mm),2-Hour serum insulin (mu U/ml),Body mass index (weight in kg/(height in m)^2),Diabetes pedigree function,Age (years),Class variable (0 or 1)
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


# SFS

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values
knn = KNeighborsClassifier(n_neighbors=2)
DT = DecisionTreeClassifier()


# Sequential Forward Selection
sfs = SFS(DT, 
          k_features=8, 
          forward=True, 
          floating=False, 
          scoring='accuracy',
          cv=8,
          n_jobs=-1)
sfs = sfs.fit(X, y)
sfs1 = sfs.k_feature_idx_

pd.DataFrame.from_dict(sfs.get_metric_dict()).T

,feature_idx,cv_scores,avg_score,feature_names,ci_bound,std_dev,std_err
1,"(1,)","[0.6666666666666666, 0.6979166666666666, 0.666...",0.703125,"(1,)",0.0268562,0.0308129,0.0116462
2,"(1, 3)","[0.6666666666666666, 0.75, 0.6145833333333334,...",0.691406,"(1, 3)",0.0444635,0.0510144,0.0192816
3,"(1, 3, 6)","[0.6354166666666666, 0.7291666666666666, 0.604...",0.679688,"(1, 3, 6)",0.0462385,0.0530509,0.0200514
4,"(1, 3, 5, 6)","[0.6875, 0.71875, 0.65625, 0.6770833333333334,...",0.699219,"(1, 3, 5, 6)",0.0254527,0.0292027,0.0110376
5,"(0, 1, 3, 5, 6)","[0.6666666666666666, 0.7083333333333334, 0.625...",0.696615,"(0, 1, 3, 5, 6)",0.0441146,0.050614,0.0191303
6,"(0, 1, 3, 5, 6, 7)","[0.6875, 0.75, 0.6354166666666666, 0.697916666...",0.717448,"(0, 1, 3, 5, 6, 7)",0.0450391,0.0516748,0.0195312
7,"(0, 1, 2, 3, 5, 6, 7)","[0.6875, 0.75, 0.6145833333333334, 0.65625, 0....",0.716146,"(0, 1, 2, 3, 5, 6, 7)",0.048628,0.0557924,0.0210875
8,"(0, 1, 2, 3, 4, 5, 6, 7)","[0.6666666666666666, 0.7604166666666666, 0.614...",0.705729,"(0, 1, 2, 3, 4, 5, 6, 7)",0.0451106,0.0517568,0.0195622


In [ ]:
sbs = SFS(DT, 
          k_features=8, 
          forward=False, 
          floating=False, 
          scoring='accuracy',
          cv=8,
          n_jobs=-1)
sbs = sbs.fit(X, y)

pd.DataFrame.from_dict(sbs.get_metric_dict()).T

,avg_score,ci_bound,cv_scores,feature_idx,feature_names,std_dev,std_err
8,0.703125,0.0418523,"[0.6770833333333334, 0.7395833333333334, 0.593...","(0, 1, 2, 3, 4, 5, 6, 7)","(0, 1, 2, 3, 4, 5, 6, 7)",0.0480185,0.0181493


In [ ]:
sffs = SFS(DT, 
           k_features=8, 
           forward=True, 
           floating=True, 
           scoring='accuracy',
           cv=8,
           n_jobs=-1)
sffs = sffs.fit(X, y)
pd.DataFrame.from_dict(sffs.get_metric_dict()).T

,feature_idx,cv_scores,avg_score,feature_names,ci_bound,std_dev,std_err
1,"(1,)","[0.6666666666666666, 0.6979166666666666, 0.666...",0.703125,"(1,)",0.0268562,0.0308129,0.0116462
2,"(1, 3)","[0.6666666666666666, 0.75, 0.6145833333333334,...",0.691406,"(1, 3)",0.0499218,0.0572769,0.0216486
3,"(1, 5, 6)","[0.7291666666666666, 0.6875, 0.677083333333333...",0.690104,"(1, 5, 6)",0.0191253,0.0219431,0.00829371
4,"(0, 1, 5, 6)","[0.6875, 0.71875, 0.6354166666666666, 0.697916...",0.704427,"(0, 1, 5, 6)",0.0387691,0.044481,0.0168122
5,"(0, 1, 5, 6, 7)","[0.6770833333333334, 0.75, 0.59375, 0.6875, 0....",0.709635,"(0, 1, 5, 6, 7)",0.0454944,0.0521971,0.0197287
6,"(0, 1, 2, 5, 6, 7)","[0.6458333333333334, 0.7916666666666666, 0.614...",0.722656,"(0, 1, 2, 5, 6, 7)",0.0529273,0.0607251,0.0229519
7,"(0, 1, 2, 3, 5, 6, 7)","[0.6875, 0.7291666666666666, 0.645833333333333...",0.714844,"(0, 1, 2, 3, 5, 6, 7)",0.0410914,0.0471455,0.0178193
8,"(0, 1, 2, 3, 4, 5, 6, 7)","[0.625, 0.7291666666666666, 0.6354166666666666...",0.703125,"(0, 1, 2, 3, 4, 5, 6, 7)",0.0433043,0.0496843,0.0187789


**เหตุผลที่ใช้ model ใน SFFS เป็น DecisionTreeClassifier เพราะใน DecisionTree มี Imformation Gain**

#งานที่2 Model Tree

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import sklearn as sk
from sklearn.neural_network import MLPClassifier
import xgboost as xgb

In [ ]:
def evaluation_accuracy(prediction,actual):
	acc = sum(prediction==actual)/len(prediction)
	return acc

#Nonnormalization

In [ ]:
max_list = [0, 1, 2, 5, 6, 7]
X = df.iloc[:,max_list] #Feature_selection_SFFS
y = df.iloc[:,-1]
X

,Number of times pregnant,Plasma glucose concentration a 2 hours in an oral glucose tolerance test,Diastolic blood pressure (mm Hg),Body mass index (weight in kg/(height in m)^2),Diabetes pedigree function,Age (years)
0,6,148,72,33.6,0.627,50
1,1,85,66,26.6,0.351,31
2,8,183,64,23.3,0.672,32
3,1,89,66,28.1,0.167,21
4,0,137,40,43.1,2.288,33
...,...,...,...,...,...,...
763,10,101,76,32.9,0.171,63
764,2,122,70,36.8,0.340,27
765,5,121,72,26.2,0.245,30
766,1,126,60,30.1,0.349,47


In [ ]:
#ทำนาย model โดยใช้ข้อมูลที่ไม่ normalization
X1_train, X1_test, y1_train, y1_test = train_test_split(X, y, test_size=0.3, random_state=1)

#DecisionTree
clf_NDT = DecisionTreeClassifier(random_state=0, max_depth=2)
clf_NDT = clf_NDT.fit(X1_train,y1_train)
y_pred_NDT = clf_NDT.predict(X1_test)
print('Accuracy DecisionTree =',evaluation_accuracy(y_pred_NDT,y1_test))
#print(confusion_matrix(y_pred_NDT, y1_test))

#GradientBoosting
clf_NGB = GradientBoostingClassifier(random_state=0)
clf_NGB = clf_NGB.fit(X1_train,y1_train)
y_pred_NGB = clf_NGB.predict(X1_test)
print('Accuracy GradientBoosting =',evaluation_accuracy(y_pred_NGB,y1_test))

#RandomForest
clf_NRF = RandomForestClassifier(max_depth=2, random_state=0)
clf_NRF = clf_NRF.fit(X1_train,y1_train)
y_pred_NRF = clf_NRF.predict(X1_test)
print('Accuracy RandomForest =',evaluation_accuracy(y_pred_NRF,y1_test))
#print(confusion_matrix(y1_test, y_pred_NRF))


#XGBoost
regressor = xgb.XGBClassifier(n_estimators=100, reg_lambda=1, gamma=0, max_depth=3)
regressor.fit(X1_train,y1_train)
y_pred_XGB = regressor.predict(X1_test)
print('Accuracy XGBoost =',evaluation_accuracy(y_pred_XGB,y1_test))

Accuracy DecisionTree = 0.7575757575757576
Accuracy GradientBoosting = 0.7705627705627706
Accuracy RandomForest = 0.7532467532467533
Accuracy XGBoost = 0.7922077922077922


#Normalization

In [ ]:
df_nor = df.copy() #df_normalization
for i in range(len(df_nor.columns)-1): #normalization
  df_nor.iloc[:,i] = (df_nor.iloc[:,i] - df_nor.iloc[:,i].min()) / (df_nor.iloc[:,i].max() - df_nor.iloc[:,i].min())
df_nor

,Number of times pregnant,Plasma glucose concentration a 2 hours in an oral glucose tolerance test,Diastolic blood pressure (mm Hg),Triceps skin fold thickness (mm),2-Hour serum insulin (mu U/ml),Body mass index (weight in kg/(height in m)^2),Diabetes pedigree function,Age (years),Class variable (0 or 1)
0,0.352941,0.743719,0.590164,0.353535,0.000000,0.500745,0.234415,0.483333,1
1,0.058824,0.427136,0.540984,0.292929,0.000000,0.396423,0.116567,0.166667,0
2,0.470588,0.919598,0.524590,0.000000,0.000000,0.347243,0.253629,0.183333,1
3,0.058824,0.447236,0.540984,0.232323,0.111111,0.418778,0.038002,0.000000,0
4,0.000000,0.688442,0.327869,0.353535,0.198582,0.642325,0.943638,0.200000,1
...,...,...,...,...,...,...,...,...,...
763,0.588235,0.507538,0.622951,0.484848,0.212766,0.490313,0.039710,0.700000,0
764,0.117647,0.613065,0.573770,0.272727,0.000000,0.548435,0.111870,0.100000,0
765,0.294118,0.608040,0.590164,0.232323,0.132388,0.390462,0.071307,0.150000,0
766,0.058824,0.633166,0.491803,0.000000,0.000000,0.448584,0.115713,0.433333,1


In [ ]:
X_nor = df_nor.iloc[:,max_list] #Feature_selection_SFFS
y_nor = df_nor.iloc[:,-1]
X_nor

,Number of times pregnant,Plasma glucose concentration a 2 hours in an oral glucose tolerance test,Diastolic blood pressure (mm Hg),Body mass index (weight in kg/(height in m)^2),Diabetes pedigree function,Age (years)
0,0.352941,0.743719,0.590164,0.500745,0.234415,0.483333
1,0.058824,0.427136,0.540984,0.396423,0.116567,0.166667
2,0.470588,0.919598,0.524590,0.347243,0.253629,0.183333
3,0.058824,0.447236,0.540984,0.418778,0.038002,0.000000
4,0.000000,0.688442,0.327869,0.642325,0.943638,0.200000
...,...,...,...,...,...,...
763,0.588235,0.507538,0.622951,0.490313,0.039710,0.700000
764,0.117647,0.613065,0.573770,0.548435,0.111870,0.100000
765,0.294118,0.608040,0.590164,0.390462,0.071307,0.150000
766,0.058824,0.633166,0.491803,0.448584,0.115713,0.433333


In [ ]:
#ทำนาย model โดยใช้ข้อมูลที่ normalization
X_train, X_test, y_train, y_test = train_test_split(X_nor, y_nor, test_size=0.3, random_state=1)

#DecisionTree
clf_DT = DecisionTreeClassifier(random_state=0, max_depth=2)
clf_DT = clf_DT.fit(X_train,y_train)
y_pred_DT = clf_DT.predict(X_test)
print('Accuracy DecisionTree =',evaluation_accuracy(y_pred_DT,y_test))

#GradientBoosting
clf_GB = GradientBoostingClassifier(random_state=0)
clf_GB = clf_GB.fit(X_train,y_train)
y_pred_GB = clf_GB.predict(X_test)
print('Accuracy GradientBoosting =',evaluation_accuracy(y_pred_GB,y_test))

#RandomForest
clf_RF = RandomForestClassifier(max_depth=2, random_state=0)
clf_RF = clf_RF.fit(X_train,y_train)
y_pred_RF = clf_RF.predict(X_test)
print('Accuracy RandomForest =',evaluation_accuracy(y_pred_RF,y_test))


#XGBoost
regressor = xgb.XGBClassifier(n_estimators=100, reg_lambda=1, gamma=0, max_depth=3)
regressor.fit(X_train,y_train)
y_pred_XGB = regressor.predict(X_test)
print('Accuracy XGBoost =',evaluation_accuracy(y_pred_XGB,y_test))

Accuracy DecisionTree = 0.7575757575757576
Accuracy GradientBoosting = 0.7705627705627706
Accuracy RandomForest = 0.7532467532467533
Accuracy XGBoost = 0.7922077922077922


# **สมาชิก**
### นายพรทวี ทวีสินธุ์ รหัส 61050246
### นายฟาริซ หะยีมะสาและ รหัส 61050258